In [1]:
from distribution import TestDistribution

from RTER import RegressionTree

import numpy as np

from sklearn.metrics import mean_squared_error as MSE
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor

from time import time
import os



In [4]:


#distribution_index_vec=[1,2,3,4]





np.random.seed(1)



sample_generator=TestDistribution(3).returnDistribution()
n_test, n_train = 1000,1000
X_train, Y_train = sample_generator.generate(n_train)
X_test, Y_test = sample_generator.generate_true(n_test)



# RTER with cv
time_start=time()
parameters={"truncate_ratio_low":[0,0.1,0.2,0.3 ,0.4], "truncate_ratio_up":[0.5, 0.6,0.7,0.8,0.9,1 ],
           "min_samples_split":[10,20,30,50,100], "max_depth":[1,2,3,4,5],
           "order":[1,2,3]}
cv_model_RTER=GridSearchCV(estimator=RegressionTree(),param_grid=parameters, cv=3, n_jobs=-1)
cv_model_RTER.fit(X_train, Y_train)

RTER_model = cv_model_RTER.best_estimator_
mse_score= - RTER_model.score(X_test, Y_test)
y_hat_RTER = RTER_model.predict(X_test)

time_end=time()

print("RTER: mse {} time {}".format(mse_score,time_end-time_start))


# GBRT
time_start=time()

model_GBRT = GradientBoostingRegressor(n_estimators = 3000)
model_GBRT.fit(X_train, Y_train.ravel())

y_hat=model_GBRT.predict(X_test)
mse_score = MSE(y_hat, Y_test)

time_end=time()

print("GBRT: mse {} time {}".format(mse_score,time_end-time_start))


# RF
time_start=time()

model_RFR = RandomForestRegressor(n_estimators = 200)
model_RFR.fit(X_train, Y_train.ravel())

y_hat=model_RFR.predict(X_test)
mse_score = MSE(y_hat, Y_test)

time_end=time()

print("RF: mse {} time {}".format(mse_score,time_end-time_start))

RTER: mse 11988.946187294368 time 10.519388675689697
GBRT: mse 299.53249948762044 time 3.242626667022705
RF: mse 389.4123095277308 time 0.5477321147918701


In [3]:
cv_model_RTER.best_params_

{'max_depth': 1,
 'min_samples_split': 50,
 'order': 1,
 'truncate_ratio_low': 0.1,
 'truncate_ratio_up': 0.9}

In [8]:
import matplotlib.pyplot as plt